In [2]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, add_accuracy_to_epochs
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MOVE ALL FUNCTIONS TO THE TOP!

In [3]:
def load_mne_objects(sub, output_name, task, LAB_root=None):
    """
    Load MNE objects for a given subject and output name.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    return {
        'HG_ev1': HG_ev1,
        'HG_base': HG_base,
        'HG_ev1_rescaled': HG_ev1_rescaled,
        'HG_ev1_evoke': HG_ev1_evoke,
        'HG_ev1_evoke_rescaled': HG_ev1_evoke_rescaled
    }

#delete this once we know the import of this function works
def add_accuracy_to_epochs(epochs, accuracy_array):
    """
    Adds accuracy data from accuracy_array to the metadata of epochs.
    Assumes the order of trials in accuracy_array matches the order in epochs.
    """
    if epochs.metadata is None:
        # Create a new DataFrame if no metadata exists
        epochs.metadata = pd.DataFrame(index=range(len(epochs)))
    
    # Ensure the accuracy_array length matches the number of epochs
    print('length of accuracy array:', len(accuracy_array))
    print('length of epochs:', len(epochs))
    assert len(accuracy_array) == len(epochs), "Mismatch in number of trials and accuracy data length."
    
    # Add the accuracy array as a new column in the metadata
    epochs.metadata['accuracy'] = accuracy_array

    # Reset the index to ensure it's sequential starting from 0
    epochs.metadata.reset_index(drop=True, inplace=True)
    
    return epochs

def create_subjects_mne_objects_dict(subjects, output_names_conditions, task, combined_data, acc_array, LAB_root=None):
    """
    Adjusted to handle multiple conditions per output name, with multiple condition columns.

    Parameters:
    - subjects: List of subject IDs.
    - output_names_conditions: Dictionary where keys are output names and values are dictionaries
        of condition column names and their required values.
    - task: Task identifier.
    - combined_data: DataFrame with combined behavioral and trial information.
    - acc_array: dict of numpy arrays of 0 for incorrect and 1 for correct trials for each subject
    - LAB_root: Root directory for data (optional).
    """
    subjects_mne_objects = {}

    for sub in subjects:
        print(f"Loading data for subject: {sub}")
        sub_mne_objects = {}
        for output_name, conditions in output_names_conditions.items():
            print(f"  Loading output: {output_name} with conditions: {conditions}")
            
            # Build the filtering condition
            sub_without_zeroes = "D" + sub[1:].lstrip('0') 
            condition_filter = (combined_data['subject_ID'] == sub_without_zeroes) # this indexes using the subject without zeroes in the name. Confusing. I know.
            
            for condition_column, condition_value in conditions.items():
                if isinstance(condition_value, list):
                    # If the condition needs to match any value in a list
                    condition_filter &= combined_data[condition_column].isin(condition_value)
                else:
                    # If the condition is a single value
                    condition_filter &= (combined_data[condition_column] == condition_value)
            
            # Filter combinedData for the specific subject and conditions
            subject_condition_data = combined_data[condition_filter]
            
            # Load MNE objects and update with accuracy data
            mne_objects = load_mne_objects(sub, output_name, task, LAB_root)
            
            if sub in acc_array:
                trial_counts = subject_condition_data['trialCount'].values.astype(int)
                accuracy_data = [acc_array[sub][i-1] for i in trial_counts if i-1 < len(acc_array[sub])] # Subtract 1 here for zero-based indexing in acc array.
                # Now pass trial_counts along with accuracy_data
                mne_objects['HG_ev1_rescaled'] = add_accuracy_to_epochs(mne_objects['HG_ev1_rescaled'], accuracy_data)

            sub_mne_objects[output_name] = mne_objects
        subjects_mne_objects[sub] = sub_mne_objects

    return subjects_mne_objects

In [4]:
# Initialize the outer dictionary.
subjects_electrodestoROIs_dict = {}

### make subjects rois to electrodes dict. Don't need to run this more than once.

In [30]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0071', 'D0077', 'D0090', 'D0100', 'D0102', 'D0103']
# subjects = ['D0103'] #testing cuz d0065 being weird

for sub in subjects:
    # sub = 'D0059'
    task = 'GlobalLocal'
    output_name = "Response_fixationCrossBase_1sec_mirror"
    events = ["Response"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False


    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    default_dict = gen_labels(good.info)
    
    # Create rawROI_dict for the subject
    rawROI_dict = defaultdict(list)
    for key, value in default_dict.items():
        rawROI_dict[value].append(key)
    rawROI_dict = dict(rawROI_dict)

    # Filter out keys containing "White-Matter"
    filtROI_dict = {key: value for key, value in rawROI_dict.items() if "White-Matter" not in key}

    # Store the dictionaries in the subjects dictionary
    subjects_electrodestoROIs_dict[sub] = {
        'default_dict': dict(default_dict),
        'rawROI_dict': dict(rawROI_dict),
        'filtROI_dict': dict(filtROI_dict)
    }


# # Save to a JSON file. Uncomment when actually running.
filename = 'subjects_electrodestoROIs_dict.json'
with open(filename, 'w') as file:
    json.dump(subjects_electrodestoROIs_dict, file, indent=4)

print(f"Saved subjects_dict to {filename}")

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0103\ieeg\sub-D0103_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


good channels before dropping bads: 225
filt channels before dropping bads: 225
outlier round 1 channels: ['LTMI8']
outlier round 2 channels: ['LTMI8', 'LAI18']
outlier round 2 channels: ['LTMI8', 'LAI18', 'LTPI1']
Bad channels in 'good': ['LTMI8', 'LAI18', 'LTPI1']
Bad channels in 'good' after dropping once: []
good channels after dropping bads: 222
filt channels after dropping bads: 222
Reading 0 ... 3198489  =      0.000 ...  1561.762 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Saved subjects_dict to subjects_electrodestoROIs_dict.json


### load subjects electrodes to rois dict

In [5]:
# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

Loaded data from subjects_electrodestoROIs_dict.json


### load high gamma data so we can do roi analysis on it
once we have more subjects, turn this into a function and loop over all subjects.  
this code is a crime against humanity

In [6]:
# Example usage
# sub = 'D0057'
# output_name = "Stimulus_i25and75_fixationCrossBase_1sec_mirror"
# task = 'GlobalLocal'
loaded_objects_D0057_i = load_mne_objects('D0057', "Stimulus_c25_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0057_c = load_mne_objects('D0057', "Stimulus_c75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# Access the objects
HG_ev1_D0057_i = loaded_objects_D0057_i['HG_ev1']
HG_base_D0057_i = loaded_objects_D0057_i['HG_base']
HG_ev1_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_rescaled']
HG_ev1_evoke_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke_rescaled']

HG_ev1_D0057_c = loaded_objects_D0057_c['HG_ev1']
HG_base_D0057_c = loaded_objects_D0057_c['HG_base']
HG_ev1_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_rescaled']
HG_ev1_evoke_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke_rescaled']

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
168 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.

load accuracy arrays so we can filter by only accurate trials  
combine this code into add_accuracy_to_epochs later!

In [7]:
# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path

# Dictionary to hold the data
acc_array = {}

# Iterate over each file in the directory
for file in os.listdir(npy_directory):
    if file.endswith('.npy'):
        # Construct the full file path
        file_path = os.path.join(npy_directory, file)
        # Load the numpy array from the file
        acc_array[file.split('_')[0]] = np.load(file_path)

# Now you have a dictionary where each key is the subject ID
# and the value is the numpy array of accuracies for that subject.

In [8]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

In [9]:
# Define a function to map blockType to congruencyProportion and switchProportion
def map_block_type(row):
    if row['blockType'] == 'A':
        return pd.Series(['25%', '25%'])
    elif row['blockType'] == 'B':
        return pd.Series(['25%', '75%'])
    elif row['blockType'] == 'C':
        return pd.Series(['75%', '25%'])
    elif row['blockType'] == 'D':
        return pd.Series(['75%', '75%'])
    else:
        return pd.Series([None, None])

# Apply the function to each row and create new columns
combined_data[['congruencyProportion', 'switchProportion']] = combined_data.apply(map_block_type, axis=1)

### load evoked and stuff for all subjects in a dictionary

In [10]:
# # example of how to use this with multiple conditions, even matching any value in a list. Although I only ever have two conditions of a type so not super necessary.
# # make sure to use the correct column names and values that match with what combinedData uses.
# output_names_conditions = {
#     "Stimulus_c25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": ["s1", "s2"]  # Example where switchType needs to match any value in the list
#     },
#     "Stimulus_i25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# congruency
# output_names = ["Stimulus_c25and75_fixationCrossBase_1sec_mirror", "Stimulus_i25and75_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_c25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#     },
#     "Stimulus_i25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#     }
# }

# switch
# output_names = ["Stimulus_r25and75_fixationCrossBase_1sec_mirror", "Stimulus_s25and75_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_r25and75_fixationCrossBase_1sec_mirror": {
#         "switchType": "r",
#     },
#     "Stimulus_s25and75_fixationCrossBase_1sec_mirror": {
#         "switchType": "s",
#     }
# }

# #  ir vs is
# output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "r"
#     },
#     "Stimulus_is_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }

# #  cr vs cs
# output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "s"
#     }
# }

# #  is vs cs
# output_names = ["Stimulus_cs_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "s"
#     },
#     "Stimulus_is_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }

# #  ir vs cr
# output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_ir_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "r"
#     }
# }

# # all interaction effects (run this with the anova code. Ugh make everything more modular later.)
# output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror", "Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]

# output_names_conditions = {
#     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "r"
#     },
#     "Stimulus_is_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     },
#     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "s"
#     }
# }

# block interaction contrasts for lwpc
output_names = ["Stimulus_c25_fixationCrossBase_1sec_mirror", "Stimulus_c75_fixationCrossBase_1sec_mirror",  \
                "Stimulus_i25_fixationCrossBase_1sec_mirror", "Stimulus_i75_fixationCrossBase_1sec_mirror"]

output_names_conditions = {
    "Stimulus_c25_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "congruencyProportion": "75%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
    },
    "Stimulus_c75_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "congruencyProportion": "25%"
    },
    "Stimulus_i25_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "congruencyProportion": "75%"
    },
    "Stimulus_i75_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "congruencyProportion": "25%"
    },
}

# block interaction contrasts for lwps
# output_names = ["Stimulus_s25_fixationCrossBase_1sec_mirror", "Stimulus_s75_fixationCrossBase_1sec_mirror",  \
#                 "Stimulus_r25_fixationCrossBase_1sec_mirror", "Stimulus_r75_fixationCrossBase_1sec_mirror"]

# output_names_conditions = {
#     "Stimulus_s25_fixationCrossBase_1sec_mirror": {
#         "switchType": "c",
#         "switchProportion": "25%"
#     },
#     "Stimulus_s75_fixationCrossBase_1sec_mirror": {
#         "switchType": "c",
#         "switchProportion": "75%"
#     },
#     "Stimulus_r25_fixationCrossBase_1sec_mirror": {
#         "switchType": "i",
#         "switchProportion": "25%"
#     },
#     "Stimulus_r75_fixationCrossBase_1sec_mirror": {
#         "switchType": "i",
#         "switchProportion": "75%"
#     },
# }

task='GlobalLocal'

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = create_subjects_mne_objects_dict(subjects, output_names_conditions, task="GlobalLocal", combined_data=combined_data, acc_array=acc_array)

Loading data for subject: D0057
  Loading output: Stimulus_c25_fixationCrossBase_1sec_mirror with conditions: {'congruency': 'c', 'congruencyProportion': '75%'}
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
168 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\

### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [11]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


### get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [12]:
def filter_electrodes_by_roi(subjects_electrodes_dict, sig_chans_per_subject, roi_list):
    """
    Filters electrodes based on specified ROIs and returns significant electrodes for each subject.

    Args:
    subjects_electrodes_dict (dict): A dictionary with subjects as keys and electrode-to-ROI mappings as values.
    sig_chans_per_subject (dict): A dictionary with subjects as keys and lists of significant channels as values.
    roi_list (list): A list of ROIs to filter electrodes.

    Returns:
    dict: A dictionary with subjects as keys and lists of significant electrodes in specified ROIs as values.
    """
    filtered_electrodes_per_subject = {}

    for sub, electrodes_dict in subjects_electrodes_dict.items():
        filtered = {key: value for key, value in electrodes_dict['filtROI_dict'].items() 
                    if any(roi in key for roi in roi_list)}

        # Aggregate electrodes into a list for each subject
        filtered_electrodes = []
        for electrodes in filtered.values():
            filtered_electrodes.extend(electrodes)

        filtered_electrodes_per_subject[sub] = filtered_electrodes
        print(f'For subject {sub}, {", ".join(roi_list)} electrodes are: {filtered_electrodes}')

    # Now filter for significant electrodes
    sig_filtered_electrodes_per_subject = {}

    for sub, filtered_electrodes in filtered_electrodes_per_subject.items():
        # Retrieve the list of significant channels for the subject
        sig_chans = sig_chans_per_subject.get(sub, [])

        # Find the intersection of filtered electrodes and significant channels for the subject
        sig_filtered_electrodes = [elec for elec in filtered_electrodes if elec in sig_chans]

        # Store the significant filtered electrodes for the subject
        sig_filtered_electrodes_per_subject[sub] = sig_filtered_electrodes
        print(f"Subject {sub} significant {', '.join(roi_list)} electrodes: {sig_filtered_electrodes}")

    return filtered_electrodes_per_subject, sig_filtered_electrodes_per_subject

# Example usage:
dlpfc_rois = ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"] #dorsolateral prefrontal cortex
acc_rois = ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"] #anterior cingulate cortex
parietal_rois = ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"] #superior parietal lobule, intraparietal sulcus, and inferior parietal lobule (split into angular gyrus and supramarginal gyrus)
dlpfc_electrodes_per_subject, sig_dlpfc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, dlpfc_rois)
acc_electrodes_per_subject, sig_acc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, acc_rois)
parietal_electrodes_per_subject, sig_parietal_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, parietal_rois)

For subject D0057, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12']
For subject D0059, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LASF10', 'LASF14', 'LASF15', 'LASF16', 'LMSF5', 'LMSF6', 'LMSF12', 'LPSF10', 'LPSF12', 'RAI10', 'RAI11', 'RAI16', 'RAMF11', 'RAMF12', 'RAMF13', 'RMMF13', 'RMMF14', 'RASF15', 'RMSF8', 'RMSF9', 'RMSF10', 'RMSF7', 'RAMF8', 'RAMF9', 'RAMF10', 'RMMF9', 'RMMF10']
For subject D0065, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RASF13', 'RASF14', 'RASF15', 'RMSF11', 'RMSF12', 'RMSF13', 'RMSF14']
For subject D0069, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: []
For subject D0071, G_fron

get total number of electrodes (make this modular with roi later once everything works)

In [11]:
total_entries = 0
for sub in sig_dlpfc_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_dlpfc_electrodes_per_subject[sub])

print("Total number of sig dlpfc electrodes across all subjects:", total_entries)

total_entries = 0
for sub in sig_acc_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_acc_electrodes_per_subject[sub])

print("Total number of sig acc electrodes across all subjects:", total_entries)

total_entries = 0
for sub in sig_parietal_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_parietal_electrodes_per_subject[sub])

print("Total number of sig parietal electrodes across all subjects:", total_entries)

Total number of sig dlpfc electrodes across all subjects: 28
Total number of sig acc electrodes across all subjects: 6
Total number of sig parietal electrodes across all subjects: 13


### do stats

current approach is to run time_perm_cluster on significant dlpfc electrodes for each subject, comparing congruent and incongruent conditions. Then, average p-values across all subjects. Discuss this with Greg, probably wrong approach.

**1/23 new approach is to average across all trials for sig dlpfc electrodes, comparing incongruent and congruent conditions. Then, run stats on this new avg electrode value x time array.

Also, I'm using HG_ev1_rescaled instead of HG_ev1 to compare congruent and incongruent, so that they're normalized with a common baseline. I think this is better than comparing the raw HG traces directly.

### this is 1/23 old approach of avg across trials first. Time perm cluster stats.

do stats and plotting together. Stats needs trial avg data, plotting just needs congruent_data without trial averaging (initially at least)  
this code is so bad right now, turn into a function later  

trialAvg is for the time perm cluster stats  
timeAvg_firstHalfSecond_firstHalfSecond_firstHalfSecond_firstHalfSecond_firstHalfSecond is for the window stats (not sure if this is even right)  



turn these into dictionaries instead of a bunch of variables later

In [14]:
# Initialize a dictionary to hold mappings
overall_electrode_mapping = []

# Initialize a dictionary to hold mappings for each ROI
electrode_mapping_per_roi = {
    'dlpfc': [],
    'acc': [],
    'parietal': []
}

# Initialize lists for storing data
output_0_data_trialAvg_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_1_data_trialAvg_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_0_data_timeAvg_firstHalfSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_1_data_timeAvg_firstHalfSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_0_data_timeAvg_secondHalfSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_1_data_timeAvg_secondHalfSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_0_data_timeAvg_fullSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_1_data_timeAvg_fullSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}

# Time windows
start_idx_firstHalfSecond, end_idx_firstHalfSecond = 2048, 3072
start_idx_secondHalfSecond, end_idx_secondHalfSecond = 3072, 4096
start_idx_fullSecond, end_idx_fullSecond = 2048, 4096


for sub in subjects:
    for roi in ['dlpfc', 'acc', 'parietal']:
        # Determine the significant electrodes for the current roi
        if roi == 'dlpfc':
            sig_electrodes = sig_dlpfc_electrodes_per_subject.get(sub, [])
        elif roi == 'acc':
            sig_electrodes = sig_acc_electrodes_per_subject.get(sub, [])
        else:  # parietal
            sig_electrodes = sig_parietal_electrodes_per_subject.get(sub, [])
        
        # Skip this roi for the current subject if no significant electrodes are present
        if not sig_electrodes:
            continue
        for electrode in sig_electrodes:
            # For each significant electrode, append a tuple to the mapping list
            # Tuple format: (Subject ID, ROI, Electrode Name, Index in List)
            # The index can be the current length of the list before appending
            index = len(overall_electrode_mapping)
            overall_electrode_mapping.append((sub, roi, electrode, index))  

            # For each significant electrode, append a tuple to the mapping list of the corresponding ROI
            # Tuple format: (Subject ID, Electrode Name, Index in List for this ROI)
            index = len(electrode_mapping_per_roi[roi])  # Get the current length of the list for this ROI
            electrode_mapping_per_roi[roi].append((sub, electrode, index))
            
        # Load trial-level data for the current condition and pick significant electrodes
        output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
        output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)

        # Calculate averages for each time window
        trial_avg_0, time_avg_0_firstHalfSecond = filter_and_average_epochs(output_0_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
        trial_avg_1, time_avg_1_firstHalfSecond = filter_and_average_epochs(output_1_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
        _, time_avg_0_secondHalfSecond = filter_and_average_epochs(output_0_epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
        _, time_avg_1_secondHalfSecond = filter_and_average_epochs(output_1_epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
        _, time_avg_0_fullSecond = filter_and_average_epochs(output_0_epochs, start_idx_fullSecond, end_idx_fullSecond)
        _, time_avg_1_fullSecond = filter_and_average_epochs(output_1_epochs, start_idx_fullSecond, end_idx_fullSecond)

        # Append the results to their respective lists
        output_0_data_trialAvg_list[roi].append(trial_avg_0)
        output_1_data_trialAvg_list[roi].append(trial_avg_1)
        output_0_data_timeAvg_firstHalfSecond_list[roi].append(time_avg_0_firstHalfSecond)
        output_1_data_timeAvg_firstHalfSecond_list[roi].append(time_avg_1_firstHalfSecond)
        output_0_data_timeAvg_secondHalfSecond_list[roi].append(time_avg_0_secondHalfSecond)
        output_1_data_timeAvg_secondHalfSecond_list[roi].append(time_avg_1_secondHalfSecond)
        output_0_data_timeAvg_fullSecond_list[roi].append(time_avg_0_fullSecond)
        output_1_data_timeAvg_fullSecond_list[roi].append(time_avg_1_fullSecond)

# After collecting all data, concatenate across subjects for each roi and condition
concatenated_trialAvg_data = {}
concatenated_timeAvg_firstHalfSecond_data = {}
concatenated_timeAvg_secondHalfSecond_data = {}
concatenated_timeAvg_fullSecond_data = {}

for roi in ['dlpfc', 'acc', 'parietal']:
    concatenated_trialAvg_data[roi] = {
        'output_0': np.concatenate(output_0_data_trialAvg_list[roi], axis=0),
        'output_1': np.concatenate(output_1_data_trialAvg_list[roi], axis=0)
    }

# Calculate mean and SEM across electrodes for all time windows and rois
overall_averages = {}
overall_sems = {}
for roi in ['dlpfc', 'acc', 'parietal']:
    overall_averages[roi] = {}
    overall_sems[roi] = {}
    for output in ['output_0', 'output_1']:
        trialAvg_data = concatenated_trialAvg_data[roi][output]
        overall_averages[roi][output] = np.nanmean(trialAvg_data, axis=0)
        overall_sems[roi][output] = np.std(trialAvg_data, axis=0, ddof=1) / np.sqrt(trialAvg_data.shape[0])



time_perm_cluster_results = {}
for roi in ['dlpfc', 'acc', 'parietal']:
    time_perm_cluster_results[roi] = time_perm_cluster(
        concatenated_trialAvg_data[roi]['output_0'],
        concatenated_trialAvg_data[roi]['output_1'], 0.05, n_jobs=6
    )

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:110: RuntimeWarning: Mean of empty slice
  time_avg_data = np.nanmean(all_epochs_data[:, :, start_idx:end_idx], axis=2)
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLo

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epo

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epo

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epo

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epo

### do window stats  
use the time avg outputs from previous cell  
use fdr correction after comparing output 0 and output 1 for each electrode to get a p-values list  

DO A SHUFFLE INSTEAD OF PAIRED T-TEST AS OF 2/7/24


shuffle test (perm test). This basically time perm cluster but avg across time.

do perm testing

In [ ]:
# Assuming the functions perform_permutation_test_within_electrodes and perform_permutation_test_across_electrodes return lists of p-values (and get loaded in properly)
p_values = {}
rois = ['dlpfc', 'acc', 'parietal']
for roi in rois:
    # Initialize p_values[roi] as a dictionary. Initialize dicts for all time windows.
    p_values[roi] = {}
    p_values[roi]['firstHalfSecond'] = {}
    p_values[roi]['secondHalfSecond'] = {}
    p_values[roi]['fullSecond'] = {}

    # Perform the tests and store results
    p_values[roi]['firstHalfSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_firstHalfSecond_list[roi], output_1_data_timeAvg_firstHalfSecond_list[roi], n_permutations=10000)
    p_values[roi]['firstHalfSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_firstHalfSecond_list[roi], output_1_data_timeAvg_firstHalfSecond_list[roi], n_permutations=10000)

    p_values[roi]['secondHalfSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_secondHalfSecond_list[roi], output_1_data_timeAvg_secondHalfSecond_list[roi], n_permutations=10000)
    p_values[roi]['secondHalfSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_secondHalfSecond_list[roi], output_1_data_timeAvg_secondHalfSecond_list[roi], n_permutations=10000)

    p_values[roi]['fullSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_fullSecond_list[roi], output_1_data_timeAvg_fullSecond_list[roi], n_permutations=10000)
    p_values[roi]['fullSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_fullSecond_list[roi], output_1_data_timeAvg_fullSecond_list[roi], n_permutations=10000)

all_p_values = {}
all_p_values['firstHalfSecond'] = []
all_p_values['secondHalfSecond'] = []
all_p_values['fullSecond'] = []

for roi in p_values:
    for test_type in p_values[roi]['firstHalfSecond']:
        p = p_values[roi]['firstHalfSecond'][test_type]
        if isinstance(p, list):
            all_p_values['firstHalfSecond'].extend(p)
        else:  # Assume it's a single float value
            all_p_values['firstHalfSecond'].append(p)

    for test_type in p_values[roi]['secondHalfSecond']:
        p = p_values[roi]['secondHalfSecond'][test_type]
        if isinstance(p, list):
            all_p_values['secondHalfSecond'].extend(p)
        else:  # Assume it's a single float value
            all_p_values['secondHalfSecond'].append(p)

    for test_type in p_values[roi]['fullSecond']:
        p = p_values[roi]['fullSecond'][test_type]
        if isinstance(p, list):
            all_p_values['fullSecond'].extend(p)
        else:  # Assume it's a single float value
            all_p_values['fullSecond'].append(p)

# Apply FDR correction
_, adjusted_p_values_firstHalfSecond = multipletests(all_p_values['firstHalfSecond'], alpha=0.05, method='fdr_bh')[:2]
_, adjusted_p_values_secondHalfSecond = multipletestsload(all_p_values['secondHalfSecond'], alpha=0.05, method='fdr_bh')[:2]
_, adjusted_p_values_fullSecond = multipletests(all_p_values['fullSecond'], alpha=0.05, method='fdr_bh')[:2]

# Incorporating adjusted p-values back into the structure is a bit more complex and depends on how you want to use them next

In [ ]:
p_values['parietal']

In [ ]:
p_values['acc']

In [ ]:
p_values['parietal']

integrate adjusted p values back in the p values dict

In [ ]:
# Step 1: Build an index map while aggregating p-values
index_map = {'firstHalfSecond': [], 'secondHalfSecond': [], 'fullSecond': []}

# Step 1: Adjusted - Ensure all p-values are treated as lists
for roi in p_values:
    for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']:
        for test_type in ['within', 'across']:
            p_value_list = p_values[roi][time_window][test_type]
            # Ensure p_value_list is actually a list
            if not isinstance(p_value_list, list):
                p_value_list = [p_value_list]
            for p_value in p_value_list:
                all_p_values[time_window].append(p_value)
                index_map[time_window].append((roi, test_type))


# Step 3: Reintegrate adjusted p-values back into the p_values structure
# Using firstHalfSecond as an example
# Adjusted reintegration example for firstHalfSecond
for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']:
    adjusted_ps = locals()[f"adjusted_p_values_{time_window}"]  # Retrieve adjusted p-values using dynamic variable names
    for i, adjusted_p in enumerate(adjusted_ps):
        roi, test_type = index_map[time_window][i]
        # Ensure the adjusted key and test_type key exist
        if 'adjusted' not in p_values[roi][time_window]:
            p_values[roi][time_window]['adjusted'] = {}
        if test_type not in p_values[roi][time_window]['adjusted']:
            p_values[roi][time_window]['adjusted'][test_type] = []
        p_values[roi][time_window]['adjusted'][test_type].append(adjusted_p)



In [ ]:
p_values['acc']

hmm figure out if need to run this always and how its different than the other one..

### do 2x2 anova for interaction effects 
this requires reloading in all four conditions (four this time cuz interaction contrasts).  
ONLY RUN THIS WHEN LOADING IN THE FOUR INTERACTION CONTRASTS RIGHT NOW.  
Integrate with other stats and plotting and stuff later.

In [13]:
# Time windows
# define these based on epoch start and also sampling rate
start_idx_firstHalfSecond, end_idx_firstHalfSecond = 2048, 3072
start_idx_secondHalfSecond, end_idx_secondHalfSecond = 3072, 4096
start_idx_fullSecond, end_idx_fullSecond = 2048, 4096

# Assuming output_names contains all four conditions
output_data_trialAvg_lists = {output_name: {'dlpfc': [], 'acc': [], 'parietal': []} for output_name in output_names}
output_data_timeAvg_firstHalfSecond_lists = {output_name: {'dlpfc': [], 'acc': [], 'parietal': []} for output_name in output_names}
output_data_timeAvg_secondHalfSecond_lists = {output_name: {'dlpfc': [], 'acc': [], 'parietal': []} for output_name in output_names}
output_data_timeAvg_fullSecond_lists = {output_name: {'dlpfc': [], 'acc': [], 'parietal': []} for output_name in output_names}

# Initialize a dictionary to hold mappings
overall_electrode_mapping = []

# Initialize a dictionary to hold mappings for each ROI
electrode_mapping_per_roi = {
    'dlpfc': [],
    'acc': [],
    'parietal': []
}

for sub in subjects:
    for roi in ['dlpfc', 'acc', 'parietal']:
        for output_name in output_names:
            # Determine significant electrodes for the current ROI and subject
            sig_electrodes = sig_dlpfc_electrodes_per_subject.get(sub, []) if roi == 'dlpfc' else sig_acc_electrodes_per_subject.get(sub, []) if roi == 'acc' else sig_parietal_electrodes_per_subject.get(sub, [])
            
            if not sig_electrodes:  # Skip if no significant electrodes
                continue
                        
            for electrode in sig_electrodes:
                # For each significant electrode, append a tuple to the mapping list
                # Tuple format: (Subject ID, ROI, Electrode Name, Index in List)
                # The index can be the current length of the list before appending
                index = len(overall_electrode_mapping)
                overall_electrode_mapping.append((sub, roi, electrode, output_name, index))  

                # For each significant electrode, append a tuple to the mapping list of the corresponding ROI
                # Tuple format: (Subject ID, Electrode Name, Index in List for this ROI)
                index = len(electrode_mapping_per_roi[roi])  # Get the current length of the list for this ROI
                electrode_mapping_per_roi[roi].append((sub, electrode, output_name, index))
                
            # Load trial-level data for the current condition and pick significant electrodes
            epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
            
            # Calculate averages for each time window
            trial_avg, time_avg_firstHalfSecond = filter_and_average_epochs(epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
            _, time_avg_secondHalfSecond = filter_and_average_epochs(epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
            _, time_avg_fullSecond = filter_and_average_epochs(epochs, start_idx_fullSecond, end_idx_fullSecond)
            
            # Append the results to their respective lists
            output_data_trialAvg_lists[output_name][roi].append(trial_avg)
            output_data_timeAvg_firstHalfSecond_lists[output_name][roi].append(time_avg_firstHalfSecond)
            output_data_timeAvg_secondHalfSecond_lists[output_name][roi].append(time_avg_secondHalfSecond)
            output_data_timeAvg_fullSecond_lists[output_name][roi].append(time_avg_fullSecond)


# After collecting all data, concatenate across subjects for each roi and condition
concatenated_trialAvg_data = {}

for roi in ['dlpfc', 'acc', 'parietal']:
    concatenated_trialAvg_data[roi] = {}
    for output_name in output_names:
        concatenated_trialAvg_data[roi][output_name] = np.concatenate(output_data_trialAvg_lists[output_name][roi], axis=0)


# Calculate mean and SEM across electrodes for all time windows and rois
overall_averages = {}
overall_sems = {}
for roi in ['dlpfc', 'acc', 'parietal']:
    overall_averages[roi] = {}
    overall_sems[roi] = {}
    for output_name in output_names:
        trialAvg_data = concatenated_trialAvg_data[roi][output_name]
        overall_averages[roi][output_name] = np.nanmean(trialAvg_data, axis=0)
        overall_sems[roi][output_name] = np.std(trialAvg_data, axis=0, ddof=1) / np.sqrt(trialAvg_data.shape[0])

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:110: RuntimeWarning: Mean of empty slice
  time_avg_data = np.nanmean(all_epochs_data[:, :, start_idx:end_idx], axis=2)
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLo

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epo

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epo

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epo

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:99: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:100: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epo

In [14]:
LAB_root = None
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Example structure for organizing data for ANOVA with four conditions
data_for_anova = []

# Function to process and append data for ANOVA from time-averaged lists
# Adapted function to include Congruency and SwitchType
def process_and_append_data_for_anova(time_averaged_lists, time_window):
    for output_name in output_names:
        # Dynamically get condition types and their values for the current output_name
        conditions = output_names_conditions[output_name]

        # # Extract congruency and switch type for the current output_name
        # congruency = output_names_conditions[output_name]["congruency"]
        # switchType = output_names_conditions[output_name]["switchType"]
        
        for roi in ['dlpfc', 'acc', 'parietal']:
            if roi == 'dlpfc':
                sig_electrodes_per_subject = sig_dlpfc_electrodes_per_subject
            elif roi == 'acc':
                sig_electrodes_per_subject = sig_acc_electrodes_per_subject
            elif roi == 'parietal':
                sig_electrodes_per_subject = sig_parietal_electrodes_per_subject

            for subject_index, subject_data in enumerate(time_averaged_lists[output_name][roi]):
                subject_id = subjects[subject_index]
                print(subject_id)

                # Skip this subject if there are no significant electrodes for them in this ROI
                if subject_id not in sig_electrodes_per_subject or not sig_electrodes_per_subject[subject_id]:
                    continue

                # Calculate the mean across trials for each electrode
                mean_activity_per_electrode = np.nanmean(subject_data, axis=0)
                # untested making this more modular 2/27
                for electrode_index, mean_activity in enumerate(mean_activity_per_electrode):
                    electrode_name = sig_electrodes_per_subject[subject_id][electrode_index]

                    # Prepare data dictionary, starting with fixed attributes
                    data_dict = {
                        'SubjectID': subject_id,
                        'Electrode': electrode_name,
                        'ROI': roi,
                        'TimeWindow': time_window,
                        'MeanActivity': mean_activity
                    }

                    # Dynamically add condition types and their values
                    data_dict.update(conditions)

                    # Append the organized data to the list
                    data_for_anova.append(data_dict)

# Invoke the function for each time-averaged list
process_and_append_data_for_anova(output_data_timeAvg_firstHalfSecond_lists, "FirstHalfSecond")
process_and_append_data_for_anova(output_data_timeAvg_secondHalfSecond_lists, "SecondHalfSecond")
process_and_append_data_for_anova(output_data_timeAvg_fullSecond_lists, "FullSecond")

# Convert to DataFrame
df_for_anova = pd.DataFrame(data_for_anova)

D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0057
D0059
D0063
D0057
D0059
D0063
D0065
D0057
D0059
D0063
D0065
D0069
D0071
D007

In [ ]:
df_for_anova

now actually run anova

In [15]:
def convert_dataframe_to_serializable_format(df):
    """
    Convert a pandas DataFrame to a serializable format that can be used with json.dump.
    """
    return df.to_dict(orient='records')

# if this works run second half second and full second too
def perform_modular_anova(df, time_window, save_dir, save_name):
    # Filter for a specific time window (I should probably make this not have a time_window input and just loop over all time windows like the within electrode code does)
    df_filtered = df[df['TimeWindow'] == time_window]

    # Dynamically construct the model formula based on condition keys
    condition_keys = [key for key in output_names_conditions[next(iter(output_names_conditions))].keys()]
    formula_terms = ' + '.join([f'C({key})' for key in condition_keys])
    interaction_terms = ' * '.join([f'C({key})' for key in condition_keys])
    formula = f'MeanActivity ~ {formula_terms} + {interaction_terms}'

    # Define the model
    model = ols(formula, data=df_filtered).fit()

    # Perform the ANOVA
    anova_results = anova_lm(model, typ=2)

    # Define the full path for the results file
    results_file_path = os.path.join(save_dir, save_name)

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(anova_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"ANOVA results saved to: {results_file_path}")

    # Print the results
    print(anova_results)

    return anova_results

In [22]:
# Example usage with the previously defined df_for_anova
perform_modular_anova(df_for_anova, "FirstHalfSecond", save_dir, 'firstHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes.txt')

ANOVA results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\firstHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes.txt
                                         sum_sq    df         F    PR(>F)
C(congruency)                          0.030755   1.0  1.030955  0.312487
C(congruencyProportion)                0.009369   1.0  0.314060  0.576505
C(congruency):C(congruencyProportion)  0.002030   1.0  0.068051  0.794754
Residual                               2.863788  96.0       NaN       NaN


,sum_sq,df,F,PR(>F)
C(congruency),0.030755,1.0,1.030955,0.312487
C(congruencyProportion),0.009369,1.0,0.314060,0.576505
C(congruency):C(congruencyProportion),0.002030,1.0,0.068051,0.794754
Residual,2.863788,96.0,NaN,NaN


In [23]:
perform_modular_anova(df_for_anova, "SecondHalfSecond", save_dir, 'SecondHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes.txt')

ANOVA results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\SecondHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes.txt
                                         sum_sq    df         F    PR(>F)
C(congruency)                          0.041490   1.0  1.484662  0.226032
C(congruencyProportion)                0.017993   1.0  0.643858  0.424299
C(congruency):C(congruencyProportion)  0.002840   1.0  0.101617  0.750590
Residual                               2.682813  96.0       NaN       NaN


,sum_sq,df,F,PR(>F)
C(congruency),0.041490,1.0,1.484662,0.226032
C(congruencyProportion),0.017993,1.0,0.643858,0.424299
C(congruency):C(congruencyProportion),0.002840,1.0,0.101617,0.750590
Residual,2.682813,96.0,NaN,NaN


In [24]:
perform_modular_anova(df_for_anova, "FullSecond", save_dir, 'fullSecond_congruency_congruencyProportion_ANOVAacrossElectrodes.txt')

ANOVA results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\fullSecond_congruency_congruencyProportion_ANOVAacrossElectrodes.txt
                                         sum_sq    df         F    PR(>F)
C(congruency)                          0.035922   1.0  1.728420  0.191745
C(congruencyProportion)                0.013332   1.0  0.641498  0.425147
C(congruency):C(congruencyProportion)  0.000017   1.0  0.000815  0.977277
Residual                               1.995177  96.0       NaN       NaN


,sum_sq,df,F,PR(>F)
C(congruency),0.035922,1.0,1.728420,0.191745
C(congruencyProportion),0.013332,1.0,0.641498,0.425147
C(congruency):C(congruencyProportion),0.000017,1.0,0.000815,0.977277
Residual,1.995177,96.0,NaN,NaN


okay now do within-electrode anova too

if the new modular code works then apply it here too (change data_for_anova basically)

In [16]:
data_for_anova = []

def process_and_append_trial_data_for_anova(time_averaged_lists, time_window, output_names_conditions):
    for output_name, conditions in output_names_conditions.items():
        
        for roi in ['dlpfc', 'acc', 'parietal']:
            if roi == 'dlpfc':
                sig_electrodes_per_subject = sig_dlpfc_electrodes_per_subject
            elif roi == 'acc':
                sig_electrodes_per_subject = sig_acc_electrodes_per_subject
            elif roi == 'parietal':
                sig_electrodes_per_subject = sig_parietal_electrodes_per_subject

            for subject_index, subject_data in enumerate(time_averaged_lists[output_name][roi]):
                subject_id = subjects[subject_index]
                
                if subject_id not in sig_electrodes_per_subject or not sig_electrodes_per_subject[subject_id]:
                    continue

                for trial_index, trial_data in enumerate(subject_data):
                    if np.any(np.isnan(trial_data)) or len(trial_data) != len(sig_electrodes_per_subject[subject_id]):
                        continue

                    for electrode_index, electrode_name in enumerate(sig_electrodes_per_subject[subject_id]):
                        activity = trial_data[electrode_index] if electrode_index < len(trial_data) else np.nan
                        
                        # Prepare the data dictionary
                        data_dict = {
                            'SubjectID': subject_id,
                            'Electrode': electrode_name,
                            'ROI': roi,
                            'TimeWindow': time_window,
                            'Trial': trial_index + 1,
                            'Activity': activity
                        }
                        
                        # Dynamically add condition types and their values
                        data_dict.update(conditions)

                        data_for_anova.append(data_dict)

# Invoke the function for each time-averaged list
process_and_append_trial_data_for_anova(output_data_timeAvg_firstHalfSecond_lists, "FirstHalfSecond", output_names_conditions)
process_and_append_trial_data_for_anova(output_data_timeAvg_secondHalfSecond_lists, "SecondHalfSecond", output_names_conditions)
process_and_append_trial_data_for_anova(output_data_timeAvg_fullSecond_lists, "FullSecond", output_names_conditions)

# Convert to DataFrame
df_for_trial_level_anova = pd.DataFrame(data_for_anova)

In [17]:
# Assuming df_for_trial_level_anova is your DataFrame and it includes a 'SubjectID' column
def perform_modular_within_electrode_anova(df, save_dir, save_name):
    results = []
    for subject_id in df['SubjectID'].unique():
        for electrode in df['Electrode'].unique():
            for time_window in df['TimeWindow'].unique():
                df_filtered = df[(df['SubjectID'] == subject_id) & 
                                 (df['Electrode'] == electrode) & 
                                 (df['TimeWindow'] == time_window)]
                
                if df_filtered.empty:
                    continue
                
                # Dynamically construct the formula based on condition keys present in the DataFrame
                condition_keys = [key for key in output_names_conditions[next(iter(output_names_conditions))].keys()]
                formula_terms = ' + '.join([f'C({key})' for key in condition_keys])
                interaction_terms = ' * '.join([f'C({key})' for key in condition_keys])
                formula = f'Activity ~ {formula_terms} + {interaction_terms}'

                # Perform the ANOVA
                model = ols(formula, data=df_filtered).fit()
                anova_results = anova_lm(model, typ=2)
                
                # Append the results
                results.append({
                    'SubjectID': subject_id,
                    'Electrode': electrode,
                    'TimeWindow': time_window,
                    'ANOVA_Results': anova_results
                })
    
    # Add the suffix '_onlySigElectrodes' to the base filename
    allElectrodesFilename = f"{save_name}_allElectrodes.txt"
    onlySigElectrodesFilename = f"{save_name}_onlySigElectrodes.txt"

    # Define the full path for the results file
    results_file_path = os.path.join(save_dir, allElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"results saved to: {results_file_path}")

    # Now process the significant results, including the subject ID in the output
    significant_results = []

    for result in results:
        anova_table = result['ANOVA_Results']
        subject_id = result['SubjectID']
        electrode = result['Electrode']
        time_window = result['TimeWindow']
        
        significant_effects = anova_table[anova_table['PR(>F)'] < 0.05]
        
        if not significant_effects.empty:
            print(f"Significant effects found for Subject: {subject_id}, Electrode: {electrode}, Time Window: {time_window}")
            print(significant_effects)
            print("\n")
            
            significant_results.append({
                'SubjectID': subject_id,
                'Electrode': electrode,
                'TimeWindow': time_window,
                'SignificantEffects': significant_effects
            })

    # Define the full path for the results file
    significant_results_file_path = os.path.join(save_dir, onlySigElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(significant_results_file_path, 'w') as file:
        file.write(significant_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_results saved to: {significant_results_file_path}")

    return results, significant_results


In [27]:
results, significant_results = perform_modular_within_electrode_anova(df_for_trial_level_anova, save_dir, 'congruency_congruencyProportion_ANOVAwithinElectrodes')

results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\congruency_congruencyProportion_ANOVAwithinElectrodes_allElectrodes.txt
Significant effects found for Subject: D0057, Electrode: RPI14, Time Window: SecondHalfSecond
                 sum_sq   df         F    PR(>F)
C(congruency)  1.899674  1.0  6.217507  0.013058


Significant effects found for Subject: D0057, Electrode: RPI14, Time Window: FullSecond
                 sum_sq   df         F    PR(>F)
C(congruency)  1.115813  1.0  7.852377  0.005325


Significant effects found for Subject: D0057, Electrode: RPIP12, Time Window: SecondHalfSecond
                 sum_sq   df        F    PR(>F)
C(congruency)  1.327977  1.0  6.71842  0.009893


Significant effects found for Subject: D0057, Electrode: RPIP12, Time Window: FullSecond
                 sum_sq   df         F    PR(>F)
C(congruency)  0.859395  1.0  8.654069  0.003454


Significant effects found for Subject: D0059, Electrode: LMMF9, Ti

In [ ]:
results

In [ ]:
significant_results 

### plot and QC stats

plot time perm cluster stats (don't run this immediately below cell if didn't do time perm cluster)

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(time_perm_cluster_results['dlpfc'])
plt.xlabel('Timepoints')
plt.ylabel('Significance (0 or 1)')
plt.title('Permutation Test Significance Over Time')
plt.show()

### plot interaction effects (only do this when load in all four of them)

In [18]:
# add the other conditions and give them condition names and colors too
plotting_parameters = {
    'Stimulus_r25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'repeat',
        'color': 'red',
        "line_style": "-"
    },
    'Stimulus_s25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'switch',
        'color': 'green',
        "line_style": "-"
    },
    'Stimulus_c25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'congruent',
        'color': 'blue',
        "line_style": "-"
    },
    'Stimulus_i25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'incongruent',
        'color': 'orange',
        "line_style": "-"
    },
    "Stimulus_ir_fixationCrossBase_1sec_mirror": {
        "condition_name": "IR",
        "color": "coral",
        "line_style": "-"
    },
    "Stimulus_is_fixationCrossBase_1sec_mirror": {
        "condition_name": "IS",
        "color": "darkseagreen",
        "line_style": "-"
    },
    "Stimulus_cr_fixationCrossBase_1sec_mirror": {
        "condition_name": "CR",
        "color": "purple",
        "line_style": "-"
    },
    "Stimulus_cs_fixationCrossBase_1sec_mirror": {
        "condition_name": "CS",
        "color": "turqoise",
        "line_style": "-"
    },
    "Stimulus_c25_fixationCrossBase_1sec_mirror": {
        "condition_name": "c25",
        "color": "red",
        "line_style": "--"
    },
    "Stimulus_c75_fixationCrossBase_1sec_mirror": {
        "condition_name": "c75",
        "color": "blue",
        "line_style": "-"
    },
    "Stimulus_i25_fixationCrossBase_1sec_mirror": {
        "condition_name": "i25",
        "color": "orange",
        "line_style": "--"
    },
    "Stimulus_i75_fixationCrossBase_1sec_mirror": {
        "condition_name": "i75",
        "color": "purple",
        "line_style": "-"
    },
    "Stimulus_s25_fixationCrossBase_1sec_mirror": {
        "condition_name": "s25",
        "color": "red",
        "line_style": "--"
    },
    "Stimulus_s75_fixationCrossBase_1sec_mirror": {
        "condition_name": "s75",
        "color": "blue",
        "line_style": "-"
    },
    "Stimulus_r25_fixationCrossBase_1sec_mirror": {
        "condition_name": "r25",
        "color": "orange",
        "line_style": "--"
    },
    "Stimulus_r75_fixationCrossBase_1sec_mirror": {
        "condition_name": "r75",
        "color": "purple",
        "line_style": "-"
    },

}

In [19]:
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

def plot_interact_effects_modular(roi, save_dir, save_name, output_names, plotting_parameters):
    # Base setup for directories and file paths
    save_path = os.path.join(save_dir, f'avg_{roi}_{save_name}_interactEffects_zscore_test.png')

    # Initialize plot
    plt.figure(figsize=(10, 6))

    # Dynamically select the first subject and use it to extract times
    first_subject_id = next(iter(subjects_mne_objects))
    example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
    times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

    overall_averages_for_plotting = {}
    overall_sem_for_plotting = {}
    # Initialize variables to store the global min and max values
    global_min_val = float('inf')  # Set to infinity initially
    global_max_val = float('-inf')  # Set to negative infinity initially
    
    # Generate labels and plot each condition
    for index, output_name in enumerate(output_names):
        # label = output_name.split("_")[1]  # OR extract label from output name instead of plotting parameters dict. Up to you.
        overall_averages_for_plotting[output_name] = overall_averages[roi][output_name]
        overall_sem_for_plotting[output_name] = overall_sems[roi][output_name]

        # # Determine line style based on index
        # line_style = '--' if index in [0, 2] else '-'  # Dotted for 1st and 3rd, solid for 0th and 2nd

        # Calculate the minimum value for this condition, including SEM
        current_min_val = min(overall_averages_for_plotting[output_name] - overall_sem_for_plotting[output_name])
        # Calculate the maximum value for this condition, including SEM
        current_max_val = max(overall_averages_for_plotting[output_name] + overall_sem_for_plotting[output_name])

        # Update the global min and max values if necessary
        global_min_val = min(global_min_val, current_min_val)
        global_max_val = max(global_max_val, current_max_val)

        # Optionally, add a small margin to the range
        margin = (global_max_val - global_min_val) * 0.05  # 5% of the range as margin
        global_min_val -= margin
        global_max_val += margin

        label = plotting_parameters[output_name]['condition_name'] # extract label from plotting parameters dict
        color = plotting_parameters[output_name]['color']
        line_style = plotting_parameters[output_name]['line_style']
        
        plt.plot(times, overall_averages_for_plotting[output_name], label=f'Average {roi} {label}', linestyle=line_style, color=color)
        plt.fill_between(times, overall_averages_for_plotting[output_name] - overall_sem_for_plotting[output_name], overall_averages_for_plotting[output_name] + overall_sem_for_plotting[output_name], alpha=0.3, color=color)

    plt.xlabel('Time (s)')
    plt.ylabel('Z-score')
    plt.title(f'Average {roi} Signal with Standard Error for {save_name}')
    plt.legend()
    # Adjust the y-axis limits
    plt.ylim([global_min_val, global_max_val])
    plt.savefig(save_path)
    plt.show()

this is just for congruent vs congruency proportion

In [31]:
plot_interact_effects_modular('dlpfc', save_dir, 'congruency_congruencyProportion', output_names, plotting_parameters)

KeyboardInterrupt: 

In [32]:
plot_interact_effects_modular('acc', save_dir, 'congruency_congruencyProportion', output_names, plotting_parameters)

KeyboardInterrupt: 

In [33]:
plot_interact_effects_modular('parietal', save_dir, 'congruency_congruencyProportion', output_names, plotting_parameters)

KeyboardInterrupt: 

### plot output 0 vs output 1 avg

this function should be able to replace all the other plotting code once it's working fully BUT only when i load in two output_names

3/5 - i think the other function for interaction effects actually replaces this one though. TEST THE OTHER FUNCTION WITH TWO OUTPUT NAMES. Also this uses hardcoded D0057...which is BAD CODING PRACTICE.

In [ ]:
def plot_avg_condition_zscore(roi, condition_0_name, condition_1_name, color_0, color_1):
    '''
    Plot output 0 and output 1 average signals against each other, with shading to indicate standard error.
    roi (str): the roi you want to plot
    condition_0_name: name of output 0 for figure and axis title (i.e., repeat)
    condition_1_name: name of output 1 for figure and axis title (i.e., switch)
    color_0: color for condition 0 signal
    color_1: color for condition 1 signal
    '''
    # Determine LAB_root based on the operating system
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
    save_path = os.path.join(save_dir, f'avg_{roi}_{condition_0_name}_VS_{condition_1_name}_zscore.png')

    plt.figure(figsize=(10, 6))

    HG_ev1_evoke_rescaled_D0057_c = subjects_mne_objects['D0057'][output_names[0]]['HG_ev1_evoke_rescaled']
    # next step is to make this flexible to number of output names, as well as use the multi-output name indexing
    output_0_avg = overall_averages[f'{roi}']['output_0']
    output_1_avg = overall_averages[f'{roi}']['output_1']
    output_0_sem = overall_sems[f'{roi}']['output_0']
    output_1_sem = overall_sems[f'{roi}']['output_1']

    # Use the times from your evoked data (assuming these are representative for all subjects)
    times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data
    

    # Determine the range of your data including SEM for better y-axis limits
    min_val = min(output_0_avg.min() - output_0_sem.max(),
                output_1_avg.min() - output_1_sem.max())

    max_val = max(output_0_avg.max() + output_0_sem.max(),
                output_1_avg.max() + output_1_sem.max())

    # Optionally, add a small margin to the range
    margin = (max_val - min_val) * 0.05  # 5% of the range as margin
    min_val -= margin
    max_val += margin

    # Your existing plotting code
    plt.figure(figsize=(10, 6))
    plt.plot(times, output_0_avg, label=f'Average {roi} {condition_0_name}', color=color_0)
    plt.fill_between(times, output_0_avg - output_0_sem, 
                    output_0_avg + output_0_sem, alpha=0.3, color=color_0)
    plt.plot(times, output_1_avg, label=f'Average {roi} {condition_1_name}', color=color_1)
    plt.fill_between(times, output_1_avg - output_1_sem, 
                    output_1_avg + output_1_sem, alpha=0.3, color=color_1)

    plt.xlabel('Time (s)')
    plt.ylabel('Z-score')
    plt.title(f'Average {roi} Signal with Standard Error ({condition_0_name} vs {condition_1_name})')
    plt.legend()

    # Adjust the y-axis limits
    plt.ylim([min_val, max_val])
    plt.savefig(save_path)  # Modify save_dir as necessary
    plt.show()


https://matplotlib.org/stable/gallery/color/named_colors.html

In [ ]:
plot_avg_condition_zscore(
    'dlpfc', 
    plotting_parameters[output_names[0]]['condition_name'], 
    plotting_parameters[output_names[1]]['condition_name'], 
    plotting_parameters[output_names[0]]['color'], 
    plotting_parameters[output_names[1]]['color']
)

In [ ]:
plot_avg_condition_zscore(
    'acc', 
    plotting_parameters[output_names[0]]['condition_name'], 
    plotting_parameters[output_names[1]]['condition_name'], 
    plotting_parameters[output_names[0]]['color'], 
    plotting_parameters[output_names[1]]['color']
)

In [ ]:
plot_avg_condition_zscore(
    'parietal', 
    plotting_parameters[output_names[0]]['condition_name'], 
    plotting_parameters[output_names[1]]['condition_name'], 
    plotting_parameters[output_names[0]]['color'], 
    plotting_parameters[output_names[1]]['color']
)

### plot individual electrodes

i think i can delete this code cuz the one for plotting interaction effects works for two output names too

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

LAB_root = None
channels = None
full_trial_base = False


if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data
firstColor, secondColor = plotting_parameters[output_names[0]]['color'], plotting_parameters[output_names[1]]['color'] # colors from plotting parameters

def plot_electrodes_grid(electrodes_data, grid_num, save_dir, roi, output_names, times, firstColor, secondColor):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        ax.plot(times, data['output_0'], label=f'{roi} {output_names[0]}', color=firstColor)
        ax.fill_between(times, 
                        data['output_0'] - np.std(data['output_0'], ddof=1) / np.sqrt(len(data['output_0'])),
                        data['output_0'] + np.std(data['output_0'], ddof=1) / np.sqrt(len(data['output_0'])), alpha=0.3)
        ax.plot(times, data['output_1'], label=f'{roi} {output_names[1]}', color=secondColor)
        ax.fill_between(times, 
                        data['output_1'] - np.std(data['output_1'], ddof=1) / np.sqrt(len(data['output_1'])),
                        data['output_1'] + np.std(data['output_1'], ddof=1) / np.sqrt(len(data['output_1'])), alpha=0.3)
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Z-score')

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{output_names[0]}_{output_names[1]}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()

# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'dlpfc'

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_dlpfc_electrodes_per_subject:
        for electrode in sig_dlpfc_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': concatenated_trialAvg_data[roi]['output_0'][electrode_counter],
                'output_1': concatenated_trialAvg_data[roi]['output_1'][electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, roi, output_names, times, firstColor, secondColor)
                
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'dlPFC', output_names, times, firstColor, secondColor)



acc

In [ ]:
# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'acc'

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_acc_electrodes_per_subject:
        for electrode in sig_acc_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': concatenated_trialAvg_data[roi]['output_0'][electrode_counter],
                'output_1': concatenated_trialAvg_data[roi]['output_1'][electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, roi, output_names, times, firstColor, secondColor)
                
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'acc', output_names, times, firstColor, secondColor)

parietal

In [ ]:
# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'parietal'
# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_parietal_electrodes_per_subject:
        for electrode in sig_parietal_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': concatenated_trialAvg_data[roi]['output_0'][electrode_counter],
                'output_1': concatenated_trialAvg_data[roi]['output_1'][electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, roi, output_names, times, firstColor, secondColor)
                
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'parietal', output_names, times, firstColor, secondColor)

### plot individual electrodes for interaction effects
i think this will just work regardless of the output names 3/5

loop this over rois instead of having another cell for each roi...
prob turn this into a function

In [21]:
LAB_root = None
channels = None
full_trial_base = False

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

# # Use the times from your evoked data (assuming these are representative for all subjects)
# times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data

def plot_electrodes_grid(electrodes_data, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        for output_name in output_names:
            color = plotting_parameters[output_name]['color']
            line_style = plotting_parameters[output_name]['line_style']
            ax.plot(times, data[output_name], label=f'{roi}_{output_name}', color=color, linestyle=line_style)
            ax.fill_between(times, 
                            data[output_name] - np.std(data[output_name], ddof=1) / np.sqrt(len(data[output_name])),
                            data[output_name] + np.std(data[output_name], ddof=1) / np.sqrt(len(data[output_name])), alpha=0.3)

        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Z-score')

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()

# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'dlpfc'
# color_map = ['red', 'green', 'blue', 'orange']
save_name = 'congruency_congruencyProportion'

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_dlpfc_electrodes_per_subject:
        for electrode in sig_dlpfc_electrodes_per_subject[sub]:
            
            electrode_data = {}
            for output_name in output_names:
                electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, roi, output_names, times, save_dir, save_name, plotting_parameters)

acc

In [ ]:
# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'acc'
save_name = 'congruency_congruencyProportion'

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_dlpfc_electrodes_per_subject:
        for electrode in sig_acc_electrodes_per_subject[sub]:
            
            electrode_data = {}
            for output_name in output_names:
                electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, roi, output_names, times, plotting_parameters, save_dir, save_name)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, roi, output_names, times, plotting_parameters, save_dir, save_name)

parietal

In [ ]:
# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'parietal'
save_name = 'congruency_congruencyProportion'

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_dlpfc_electrodes_per_subject:
        for electrode in sig_parietal_electrodes_per_subject[sub]:
            
            electrode_data = {}
            for output_name in output_names:
                electrode_data[output_name] = concatenated_trialAvg_data[roi][output_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, roi, output_names, times, plotting_parameters, save_dir, save_name)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, roi, output_names, times, plotting_parameters, save_dir, save_name)
